In [1]:
import scipy.io as sio
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt

In [2]:
# Load data, the data used in this script is Hand-written letters data set.
# The object 'data' is a dic that contains np.array 'train_x' and 'train_y'.
matfn='letters_data.mat'
data=sio.loadmat(matfn)

In [3]:
labels=np.zeros(shape=(len(data['train_y']),26))
for i in range(len(data['train_y'])):
    labels[i,(data['train_y'][i][0]-1)]=1

In [4]:
False in [sum(i)==1 for i in labels]

False

In [5]:
# Normalize data.
mean=data['train_x'].mean(0)
sd=data['train_x'].std(0)
for i in range(len(sd)):
    if sd[i]==0:
        sd[i]=1

In [6]:
train_x=(data['train_x']-mean)/sd

In [7]:
# Split the training set and validation set.
index=np.arange(len(train_x))
np.random.shuffle(index)
train_x=train_x[index]
labels=labels[index]
train_set=train_x[0:round(len(train_x)*0.8)]
train_label=labels[0:round(len(train_x)*0.8)]
valid_set=train_x[round(len(train_x)*0.8):]
valid_label=labels[round(len(train_x)*0.8):]

In [8]:
def add_1(data):
    return np.array(list(data.transpose())+[np.repeat(1,len(data))]).transpose()
def add_1_vector(vector):
    return np.matrix(vector.T.tolist()[0]+[1]).T

In [9]:
train_set=add_1(train_set)
valid_set=add_1(valid_set)

In [10]:
def trainNeuralNetwork(train_set,train_label,epoch=10,V=None,W=None,epsilon1=0.02,epsilon2=0.005,eta=10,d1=0.8,d2=0.8):
    if V==None:
        V=np.random.normal(scale=(1/eta)**2,size=500*785)
        V=np.matrix(V.reshape((500,785)))
    if W==None:
        W=np.random.normal(scale=(1/eta)**2,size=26*501)
        W=np.matrix(W.reshape((26,501)))
    for i in range(epoch):
        index=np.arange(len(train_set))
        np.random.shuffle(index)
        train_set=train_set[index]
        train_label=train_label[index]
        print(i)
        for j in range(int(len(train_set)/80)):
            x=np.matrix(train_set[(j*80):((j+1)*80)]).T
            y=np.matrix(train_label[(j*80):((j+1)*80)]).T
            h=np.tanh(V*x)
            h=np.matrix(list(np.array(h))+[np.repeat(1,80)])
            z=1/(1+np.exp(-W*h))
            z_y=z-y
            L_h=W.T*z_y
            grad_W=np.matrix(np.zeros((26,501)))
            grad_V=np.matrix(np.zeros((500,785)))
            for k in range(80):
                grad_W=grad_W+z_y[...,k]*h[...,k].T
                grad_V=grad_V+np.matrix(np.array(L_h[...,k][:-1])*(1-np.array(h[...,k][:-1])*np.array(h[...,k][:-1])))*x[...,k].T
            W=W-epsilon2*grad_W
            V=V-epsilon1*grad_V
        epsilon1=epsilon1*d1
        epsilon2=epsilon2*d2
    return {'W':W,'V':V}

In [11]:
def predictNeuralNetwork(valid_set,weights):
    V=weights['V']
    W=weights['W']
    X=np.matrix(valid_set).T
    h=np.tanh(V*X)
    h=np.matrix(list(np.array(h))+[np.repeat(1,len(valid_set))])
    z=1/(1+np.exp(-W*h))
    return np.array(z.T)

In [12]:
def encode(labels):
    res=[]
    for i in range(len(labels)):
        res.append(labels[i].argmax()+1)
    return np.array(res)

In [42]:
weight1=trainNeuralNetwork(train_set,train_label,epoch=5,epsilon1=0.001,epsilon2=0.003)

0
1
2
3
4


In [43]:
pred1=predictNeuralNetwork(valid_set,weight1)

In [44]:
# Get predict accuracy
sum(encode(pred1)==encode(valid_label))/len(valid_label)

0.89202724358974361